In [4]:
# Desinstalar el SDK viejo
%pip uninstall -y alpaca-trade-api

# Instalar el SDK moderno y yfinance
%pip install alpaca-py yfinance

# Instalar dependencias adicionales
%pip install numpy pandas scipy matplotlib requests

In [13]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from alpaca.data import StockHistoricalDataClient  # NUEVO SDK
from alpaca.data.requests import StockBarsRequest   # NUEVO
from alpaca.data.timeframe import TimeFrame         # NUEVO
from scipy import signal
from scipy.signal import find_peaks
from scipy.stats import linregress
import matplotlib.pyplot as plt  # Para visualización opcional
from typing import Optional, Tuple
import requests
import yfinance as yf

# ==================== CONFIGURACIÓN ALPACA ====================
API_KEY = os.getenv("ALPACA_API_KEY", "PK6PR8K71SUI4RGI2JMB")
API_SECRET = os.getenv("ALPACA_API_SECRET", "2t6mK57kzfZNFJKVKIAAEI7Grvj0rOrSajwnVXvq")

# NUEVO: Inicializar cliente moderno (no necesita base_url)
alpaca_client = StockHistoricalDataClient(API_KEY, API_SECRET)

def get_yfinance_data(ticker: str, start_date: str, end_date: str, timeframe: str = "1Day") -> pd.DataFrame:
    """
    Descarga datos históricos usando yfinance como fallback.
    """
    if timeframe == "1Day":
        interval = "1d"
    elif timeframe == "1Hour":
        interval = "1h"
    else:
        raise ValueError("Solo se soporta '1Day' o '1Hour'")

    df = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    if df.empty:
        raise ValueError(f"Sin datos de yfinance para {ticker}")

    # Normalizar columnas
    df = df.rename(columns={
        "Adj Close": "Adj Close"
    })
    df.index = pd.to_datetime(df.index)
    return df

def get_alpaca_data(ticker: str, start_date: str, end_date: str, timeframe: str = "1Day") -> pd.DataFrame:
    """
    Descarga datos históricos usando alpaca-py (SDK moderno), con fallback a yfinance si falla.
    """
    try:
        # Mapear timeframe
        if timeframe == "1Day":
            tf = TimeFrame.Day
        elif timeframe == "1Hour":
            tf = TimeFrame.Hour
        else:
            raise ValueError("Solo se soporta '1Day' o '1Hour'")

        # Crear request
        request_params = StockBarsRequest(
            symbol_or_symbols=ticker,
            timeframe=tf,
            start=pd.Timestamp(start_date).date(),
            end=pd.Timestamp(end_date).date()
        )

        # Obtener datos
        bars = alpaca_client.get_stock_bars(request_params)

        if not bars or ticker not in bars.data:
            raise ValueError(f"Sin datos de Alpaca para {ticker}")

        # Convertir a DataFrame
        df = bars.df.reset_index()
        df = df[df['symbol'] == ticker].set_index('timestamp')

        # Normalizar columnas
        df = df.rename(columns={
            "open": "Open", "high": "High", "low": "Low",
            "close": "Close", "volume": "Volume"
        })

        df.index = pd.to_datetime(df.index)
        df["Adj Close"] = df["Close"]
        return df
    except Exception as e:
        print(f"Error con Alpaca para {ticker}: {e}. Usando yfinance como fallback.")
        return get_yfinance_data(ticker, start_date, end_date, timeframe)

def resample_to_weekly(df: pd.DataFrame) -> pd.DataFrame:
    """
    Resamplea datos diarios a semanales.
    """
    df_weekly = df.resample('W').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Adj Close': 'last',
        'Volume': 'sum'
    })
    return df_weekly

# ==================== PARÁMETROS GLOBALES ====================
TICKERS = ["AAPL", "META", "AMZN", "MSFT", "GOOGL", "TSLA", "NVDA", "GE", "HD", "CAT", "WMT"]
START_DATE = "2023-01-01"
END_DATE   = "2025-11-01"

# ==================== INDICADORES ====================
def EMA(series: pd.Series, period: int) -> pd.Series:
    return series.ewm(span=period, adjust=False).mean()

def calcular_SMA(series: pd.Series, period: int) -> pd.Series:
    return series.rolling(window=period).mean()

def calcular_MACD(close_series: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
    ema_fast = EMA(close_series, fast)
    ema_slow = EMA(close_series, slow)
    macd_line = ema_fast - ema_slow
    signal_line = EMA(macd_line, signal)
    histogram = macd_line - signal_line
    macd_slope = macd_line.diff()
    return macd_line, signal_line, histogram, macd_slope

def calcular_RSI_Wilder(close_series: pd.Series, period: int = 14) -> pd.Series:
    delta = close_series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calcular_bollinger_bands(series: pd.Series, period: int = 20, std_dev: float = 2.0) -> Tuple[pd.Series, pd.Series, pd.Series]:
    middle = series.rolling(window=period).mean()
    std = series.rolling(window=period).std()
    upper = middle + (std * std_dev)
    lower = middle - (std * std_dev)
    return upper, middle, lower

# ==================== MOTOR DE PATRONES ====================
def get_picos_valles(series: pd.Series, order_n: int, prominence: float = 0.01) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    picos_idx = find_peaks(series.values, distance=order_n, prominence=prominence)[0]
    valles_idx = find_peaks(-series.values, distance=order_n, prominence=prominence)[0]
    return picos_idx, series.iloc[picos_idx].values, valles_idx, series.iloc[valles_idx].values

def get_trendline(indices: np.ndarray, values: np.ndarray) -> Optional[Tuple[float, float]]:
    if len(indices) < 2:
        return None
    res = linregress(indices, values)
    return res.slope, res.intercept

def is_near(val1: float, val2: float, tolerance_pct: float = 0.03) -> bool:
    if val2 == 0: return abs(val1) <= tolerance_pct
    return abs(val1 - val2) / abs(val2) <= tolerance_pct

# --- Definiciones de Patrones ---
def detectar_hch(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, tolerance_pct=0.03, vol_multiplier=1.5) -> str:
    if len(picos_idx) < 3 or len(valles_idx) < 2: return "NEUTRAL"
    P1_idx, P2_idx, P3_idx = picos_idx[-3:]
    P1_val, P2_val, P3_val = picos_val[-3:]
    V1_idx, V2_idx = valles_idx[-2:]
    V1_val, V2_val = valles_val[-2:]
    cond1 = (P2_val > P1_val) and (P2_val > P3_val)
    cond2 = is_near(P1_val, P3_val, tolerance_pct + 0.02)
    cond3 = is_near(V1_val, V2_val, tolerance_pct)
    cond4 = (V1_idx > P1_idx) and (V1_idx < P2_idx)
    cond5 = (V2_idx > P2_idx) and (V2_idx < P3_idx)
    if cond1 and cond2 and cond3 and cond4 and cond5:
        neckline_level = (V1_val + V2_val) / 2
        if current_price < neckline_level:
            avg_vol = data_slice['Volume'].mean()
            recent_vol = data_slice['Volume'].iloc[-1]
            if recent_vol > avg_vol * vol_multiplier:
                return "REVERSION_BAJISTA_HCH"
    return "NEUTRAL"

def detectar_hch_invertido(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, tolerance_pct=0.03, vol_multiplier=1.5) -> str:
    if len(valles_idx) < 3 or len(picos_idx) < 2: return "NEUTRAL"
    V1_idx, V2_idx, V3_idx = valles_idx[-3:]
    V1_val, V2_val, V3_val = valles_val[-3:]
    P1_idx, P2_idx = picos_idx[-2:]
    P1_val, P2_val = picos_val[-2:]
    cond1 = (V2_val < V1_val) and (V2_val < V3_val)
    cond2 = is_near(V1_val, V3_val, tolerance_pct + 0.02)
    cond3 = is_near(P1_val, P2_val, tolerance_pct)
    cond4 = (P1_idx > V1_idx) and (P1_idx < V2_idx)
    cond5 = (P2_idx > V2_idx) and (P2_idx < V3_idx)
    if cond1 and cond2 and cond3 and cond4 and cond5:
        neckline_level = (P1_val + P2_val) / 2
        if current_price > neckline_level:
            avg_vol = data_slice['Volume'].mean()
            recent_vol = data_slice['Volume'].iloc[-1]
            if recent_vol > avg_vol * vol_multiplier:
                return "REVERSION_ALCISTA_HCH_INVERTIDO"
    return "NEUTRAL"

def detectar_doble_techo(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, tolerance_pct=0.03, vol_multiplier=1.5) -> str:
    if len(picos_idx) < 2 or len(valles_idx) < 1: return "NEUTRAL"
    P1_val, P2_val = picos_val[-2:]
    P1_idx, P2_idx = picos_idx[-2:]
    V1_val = valles_val[-1]
    V1_idx = valles_idx[-1]
    cond1 = is_near(P1_val, P2_val, tolerance_pct)
    cond2 = (V1_idx > P1_idx) and (V1_idx < P2_idx)
    if cond1 and cond2:
        neckline_level = V1_val
        if current_price < neckline_level:
            avg_vol = data_slice['Volume'].mean()
            recent_vol = data_slice['Volume'].iloc[-1]
            # Regla de Validación por Volumen: Para Doble Techo, gana fuerza si volumen en segundo pico < primero
            vol_p1 = data_slice['Volume'].iloc[P1_idx]
            vol_p2 = data_slice['Volume'].iloc[P2_idx]
            volume_strength = vol_p2 < vol_p1
            if recent_vol > avg_vol * vol_multiplier and volume_strength:
                return "REVERSION_BAJISTA_DOBLE_TECHO"
            else:
                return "NEUTRAL"  # Si no confirma volumen, no detectar patrón
    return "NEUTRAL"

def detectar_doble_valle(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, tolerance_pct=0.03, vol_multiplier=1.5) -> str:
    if len(valles_idx) < 2 or len(picos_idx) < 1: return "NEUTRAL"
    V1_val, V2_val = valles_val[-2:]
    V1_idx, V2_idx = valles_idx[-2:]
    P1_val = picos_val[-1]
    P1_idx = picos_idx[-1]
    cond1 = is_near(V1_val, V2_val, tolerance_pct)
    cond2 = (P1_idx > V1_idx) and (P1_idx < V2_idx)
    if cond1 and cond2:
        neckline_level = P1_val
        if current_price > neckline_level:
            avg_vol = data_slice['Volume'].mean()
            recent_vol = data_slice['Volume'].iloc[-1]
            # Para Doble Valle, no especifica volumen entre valles, pero usamos confirmación general
            if recent_vol > avg_vol * vol_multiplier:
                return "REVERSION_ALCISTA_DOBLE_VALLE"
            else:
                return "NEUTRAL"  # Si no confirma volumen, no detectar patrón
    return "NEUTRAL"

def detectar_triangulo_ascendente(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, tolerance_pct=0.03, vol_multiplier=1.5, min_points=3) -> str:
    if len(picos_idx) < min_points or len(valles_idx) < min_points: return "NEUTRAL"
    resistencia = get_trendline(picos_idx[-min_points:], picos_val[-min_points:])
    soporte = get_trendline(valles_idx[-min_points:], valles_val[-min_points:])
    if resistencia and soporte:
        res_slope, res_intercept = resistencia
        sup_slope, sup_intercept = soporte
        cond1 = (abs(res_slope) < 0.05)
        cond2 = (sup_slope > 0.1)
        if cond1 and cond2:
            resistencia_level = picos_val[-1]
            if current_price > resistencia_level:
                avg_vol = data_slice['Volume'].mean()
                recent_vol = data_slice['Volume'].iloc[-1]
                if recent_vol > avg_vol * vol_multiplier:
                    return "CONTINUACION_ALCISTA_TRIANGULO_ASC"
                else:
                    return "NEUTRAL"
    return "NEUTRAL"

def detectar_triangulo_descendente(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, tolerance_pct=0.03, vol_multiplier=1.5, min_points=3) -> str:
    if len(picos_idx) < min_points or len(valles_idx) < min_points: return "NEUTRAL"
    resistencia = get_trendline(picos_idx[-min_points:], picos_val[-min_points:])
    soporte = get_trendline(valles_idx[-min_points:], valles_val[-min_points:])
    if resistencia and soporte:
        res_slope, res_intercept = resistencia
        sup_slope, sup_intercept = soporte
        cond1 = (res_slope < -0.1)
        cond2 = (abs(sup_slope) < 0.05)
        if cond1 and cond2:
            soporte_level = valles_val[-1]
            if current_price < soporte_level:
                avg_vol = data_slice['Volume'].mean()
                recent_vol = data_slice['Volume'].iloc[-1]
                if recent_vol > avg_vol * vol_multiplier:
                    return "CONTINUACION_BAJISTA_TRIANGULO_DESC"
                else:
                    return "NEUTRAL"
    return "NEUTRAL"

def detectar_banderas_banderolas(data_slice, current_price, lookback_asta=20, lookback_cons=7, min_move_pct=0.10, vol_multiplier=1.5) -> str:
    if len(data_slice) < lookback_asta: return "NEUTRAL"
    slice_asta = data_slice.iloc[-lookback_asta:]
    min_asta = slice_asta['Low'].min()
    max_asta = slice_asta['High'].max()
    if (max_asta - min_asta) / min_asta > min_move_pct:
        consolidacion = data_slice.iloc[-lookback_cons:]
        trend = get_trendline(np.arange(len(consolidacion)), consolidacion['Close'].values)
        if trend:
            slope, _ = trend
            if abs(slope) < 0.1:
                if current_price > max_asta:
                    avg_vol = data_slice['Volume'].mean()
                    recent_vol = data_slice['Volume'].iloc[-1]
                    if recent_vol > avg_vol * vol_multiplier:
                        return "CONTINUACION_ALCISTA_BANDERA"
                    else:
                        return "NEUTRAL"
    return "NEUTRAL"

# --- Función de Visualización (Opcional) ---
def plot_patrones(data_slice: pd.DataFrame, picos_idx, picos_val, valles_idx, valles_val, detected_pattern: str):
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(data_slice.index, data_slice['Close'], label='Close Price')
    if len(picos_idx):
        ax.scatter(data_slice.index[picos_idx], picos_val, marker='^', label='Picos')
    if len(valles_idx):
        ax.scatter(data_slice.index[valles_idx], valles_val, marker='v', label='Valles')
    if 'TRIANGULO' in detected_pattern:
        res_idx = picos_idx[-3:] if len(picos_idx) >= 3 else picos_idx
        res_val = picos_val[-3:] if len(picos_val) >= 3 else picos_val
        sup_idx = valles_idx[-3:] if len(valles_idx) >= 3 else valles_idx
        sup_val = valles_val[-3:] if len(valles_val) >= 3 else valles_val
        res = get_trendline(res_idx, res_val) if len(res_idx) >= 2 else None
        sup = get_trendline(sup_idx, sup_val) if len(sup_idx) >= 2 else None
        if res:
            ax.plot(data_slice.index[res_idx], res[0] * np.array(res_idx) + res[1], label='Resistencia')
        if sup:
            ax.plot(data_slice.index[sup_idx], sup[0] * np.array(sup_idx) + sup[1], label='Soporte')
    ax.set_title(f'Patrón Detectado: {detected_pattern}')
    ax.legend()
    plt.show()

# --- Motor de Patrones (El que llama a las funciones) ---
PATRONES_PRIORIDAD = [
    (detectar_hch, "picos"),
    (detectar_hch_invertido, "picos"),
    (detectar_doble_techo, "picos"),
    (detectar_doble_valle, "picos"),
    (detectar_triangulo_ascendente, "picos"),
    (detectar_triangulo_descendente, "picos"),
    (detectar_banderas_banderolas, "data")
]

def analizar_patrones_recientes(
    data_slice: pd.DataFrame,
    current_price: float,
    order_n: int = 5,
    prominence: float = 0.01,
    tolerance_pct: float = 0.03,
    vol_multiplier: float = 1.5,
    min_points_trend: int = 3,
    lookback_asta: int = 20,
    lookback_cons: int = 7,
    min_move_pct: float = 0.10,
    plot: bool = False
) -> str:
    if data_slice.empty: return "NEUTRAL"
    picos_idx, picos_val, valles_idx, valles_val = get_picos_valles(
        data_slice['Close'], order_n, prominence
    )
    for detector_func, tipo_input in PATRONES_PRIORIDAD:
        kwargs = {'vol_multiplier': vol_multiplier, 'data_slice': data_slice}
        name = detector_func.__name__
        if 'triangulo' in name:
            kwargs.update({'tolerance_pct': tolerance_pct, 'min_points': min_points_trend})
        elif 'doble' in name or 'hch' in name:
            kwargs.update({'tolerance_pct': tolerance_pct})
        elif 'banderas' in name:
            kwargs.update({'lookback_asta': lookback_asta, 'lookback_cons': lookback_cons, 'min_move_pct': min_move_pct})
        if tipo_input == "data":
            estado = detector_func(data_slice, current_price, **{k:v for k,v in kwargs.items() if k != 'data_slice'})
        else:
            estado = detector_func(picos_idx, picos_val, valles_idx, valles_val, current_price, data_slice, **{k:v for k,v in kwargs.items() if k != 'data_slice'})
        if estado != "NEUTRAL":
            if plot:
                plot_patrones(data_slice, picos_idx, picos_val, valles_idx, valles_val, estado)
            return estado
    return "NEUTRAL"

# ==================== DETECCIÓN DE ONDAS DE ELLIOTT ====================
def encontrar_pivotes(df, porcentaje_cambio=0.05):
    if 'Close' not in df.columns:
        raise ValueError("El DataFrame debe tener la columna 'Close'.")
    data_y = df['Close'].values
    timestamps = df.index
    peak_indexes = signal.argrelextrema(data_y, np.greater)[0]
    peaks = pd.DataFrame({'idx_num': peak_indexes, 'price': data_y[peak_indexes], 'type': 'high'})
    valley_indexes = signal.argrelextrema(data_y, np.less)[0]
    valleys = pd.DataFrame({'idx_num': valley_indexes, 'price': data_y[valley_indexes], 'type': 'low'})
    all_pivots = pd.concat([peaks, valleys]).sort_values('idx_num').reset_index(drop=True)
    if all_pivots.empty:
        return pd.DataFrame(columns=['timestamp', 'price', 'type'])
    filtered = [all_pivots.iloc[0]]
    previous_price = all_pivots['price'].iloc[0]
    for i in range(1, len(all_pivots)):
        current_price = all_pivots['price'].iloc[i]
        rel_diff = abs(current_price - previous_price) / previous_price if previous_price != 0 else 0
        if rel_diff >= porcentaje_cambio:
            filtered.append(all_pivots.iloc[i])
            previous_price = current_price
    if not filtered:
         return pd.DataFrame(columns=['timestamp', 'price', 'type'])
    filtered_pivots = pd.concat(filtered, axis=1).T
    filtered_pivots['timestamp'] = timestamps[filtered_pivots['idx_num'].values.astype(int)]
    filtered_pivots['type'] = filtered_pivots['type'].replace({'high':'high','low':'low'})
    return filtered_pivots[['timestamp', 'price', 'type']]

def validar_impulso_elliott(pivotes, usar_fibonacci=True):
    if len(pivotes) < 6:
        return False, None
    for i in range(len(pivotes) - 5):
        seq = pivotes.iloc[i:i+6]
        expected_types = ['low', 'high', 'low', 'high', 'low', 'high']
        if seq['type'].tolist() != expected_types:
            continue
        P0, P1, P2, P3, P4, P5 = seq['price'].values
        if P2 <= P0:
            continue
        wave1 = P1 - P0
        wave3 = P3 - P2
        wave5 = P5 - P4
        if wave3 < wave1 and wave3 < wave5:
            continue
        if P4 <= P1:
            continue
        if usar_fibonacci and wave3 < 1.618 * wave1:
            continue
        return True, seq
    return False, None

# ==================== INTERPRETACIÓN DE SEÑALES ====================
def interpretar_rsi(rsi_value: float) -> str:
    if rsi_value > 70:
        return "overbought"
    elif 65 <= rsi_value < 70:
        return "caution_alcista"
    elif 35 <= rsi_value < 65:
        return "neutral"
    elif 30 <= rsi_value < 35:
        return "caution_bajista"
    elif rsi_value < 30:
        return "oversold"
    return "neutral"

def interpretar_macd(macd_value: float, signal_value: float) -> str:
    if macd_value > signal_value:
        return "buy"
    elif macd_value < signal_value:
        return "sell"
    else:
        return "neutral"

def interpretar_ma(close: float, sma50: float, sma200: float) -> str:
    if sma50 is None or sma200 is None or sma50 == 0 or sma200 == 0:
        return "neutral"
    if close > sma50 and sma50 > sma200:
        return "buy"
    elif close < sma50 and sma50 < sma200:
        return "sell"
    else:
        return "neutral"

def interpretar_cross(sma50_series: pd.Series, sma200_series: pd.Series) -> str:
    if sma50_series is None or sma200_series is None:
        return "neutral"
    if len(sma50_series.dropna()) < 2 or len(sma200_series.dropna()) < 2:
        return "neutral"
    if sma50_series.iloc[-2] < sma200_series.iloc[-2] and sma50_series.iloc[-1] > sma200_series.iloc[-1]:
        return "buy"
    elif sma50_series.iloc[-2] > sma200_series.iloc[-2] and sma50_series.iloc[-1] < sma200_series.iloc[-1]:
        return "sell"
    else:
        return "neutral"

def interpretar_bollinger(close: float, upper: float, lower: float) -> str:
    if close > upper:
        return "sell"
    elif close < lower:
        return "buy"
    else:
        return "neutral"

# ==================== LÓGICA DE DECISIÓN ====================
def get_decision(patron: str, es_elliott_valido: bool, cross_sig: str, ma_sig: str, macd_last: float, signal_last: float, rsi_last: float, close: float, upper_bb: float, lower_bb: float) -> str:
    patron_sig = "neutral"
    if patron != "NEUTRAL":
        if "ALCISTA" in patron:
            patron_sig = "buy"
        elif "BAJISTA" in patron:
            patron_sig = "sell"
    if es_elliott_valido:
        patron_sig = "buy"
    if cross_sig != "neutral":
        patron_sig = cross_sig
    if patron_sig != "neutral":
        return patron_sig if patron_sig in ["buy", "sell"] else "hold"
    trend = ma_sig
    if trend == "neutral":
        return "hold"
    histogram_last = macd_last - signal_last
    threshold = 0.25
    macd_strong_above = histogram_last > threshold
    macd_weak_above = 0 < histogram_last <= threshold
    macd_strong_below = histogram_last < -threshold
    macd_weak_below = -threshold <= histogram_last < 0
    overbought_rsi = rsi_last > 70
    oversold_rsi = rsi_last < 30
    caution_alcista = 65 <= rsi_last < 70
    caution_bajista = 30 <= rsi_last < 35
    overbought_bb = close > upper_bb
    oversold_bb = close < lower_bb
    overbought = overbought_rsi or overbought_bb
    oversold = oversold_rsi or oversold_bb
    if trend == "buy":
        if macd_strong_above:
            if overbought:
                return "hold"
            elif caution_alcista:
                return "hold"
            else:
                return "buy"
        elif macd_weak_above:
            return "hold"
        else:
            if overbought:
                return "sell"
            else:
                return "hold"
    elif trend == "sell":
        if macd_strong_below:
            if oversold:
                return "hold"
            elif caution_bajista:
                return "hold"
            else:
                return "sell"
        elif macd_weak_below:
            return "hold"
        else:
            if oversold:
                return "buy"
            else:
                return "hold"
    return "hold"

# ==================== NUEVAS FUNCIONES: RECOMENDACIÓN DE OPCIONES ====================
def get_weekly_options(ticker, current_date, close_last):
    """
    Fallback directo a yfinance (no usa Alpaca para opciones)
    """
    current_dt = pd.to_datetime(current_date)
    target_date = current_dt + pd.Timedelta(days=7)
    days_ahead = 4 - target_date.weekday()
    if days_ahead <= 0:
        days_ahead += 7
    expiration_date = target_date + pd.Timedelta(days=days_ahead)
    expiration_str = expiration_date.strftime('%Y-%m-%d')

    try:
        stock = yf.Ticker(ticker)
        exp_dates = stock.options

        if not exp_dates:
            return pd.DataFrame(), pd.DataFrame()

        # Buscar expiración más cercana
        exp_date_str = None
        for date_str in exp_dates:
            exp_dt = pd.to_datetime(date_str)
            if exp_dt >= expiration_date:
                exp_date_str = date_str
                break

        if not exp_date_str:
            exp_date_str = exp_dates[0]  # fallback a la primera disponible

        # Obtener cadena de opciones
        opt_chain = stock.option_chain(exp_date_str)

        calls = opt_chain.calls.copy()
        puts = opt_chain.puts.copy()

        # Normalizar columnas
        calls['type'] = 'call'
        calls['expiration_date'] = exp_date_str
        calls['close'] = calls.get('lastPrice', calls.get('last', 0))

        puts['type'] = 'put'
        puts['expiration_date'] = exp_date_str
        puts['close'] = puts.get('lastPrice', puts.get('last', 0))

        # Filtrar strikes cercanos
        calls = calls[(calls['strike'] >= 0.8 * close_last) & (calls['strike'] <= 1.2 * close_last)]
        puts = puts[(puts['strike'] >= 0.8 * close_last) & (puts['strike'] <= 1.2 * close_last)]

        return calls, puts

    except Exception as e:
        print(f"    Error con yfinance: {e}")
        return pd.DataFrame(), pd.DataFrame()

def get_option_recommendation(decision, calls, puts, current_price):
    """
    Determina la recomendación de opción basada en la señal de trading.
    """
    if decision == 'buy':
        if calls.empty:
            return "Compra Call Weekly - No hay contratos disponibles"

        atm_call_idx = (calls['strike'] - current_price).abs().idxmin()
        atm_call = calls.loc[atm_call_idx]
        strike = atm_call['strike']
        premium = atm_call.get('close', 0)
        expiration = atm_call.get('expiration_date', 'N/A')

        return f"Compra Call Weekly - Strike: {strike:.2f}, Expiración: {expiration}, Prima: {premium:.2f}"

    elif decision == 'sell':
        if puts.empty:
            return "Compra Put Weekly - No hay contratos disponibles"

        atm_put_idx = (puts['strike'] - current_price).abs().idxmin()
        atm_put = puts.loc[atm_put_idx]
        strike = atm_put['strike']
        premium = atm_put.get('close', 0)
        expiration = atm_put.get('expiration_date', 'N/A')

        return f"Compra Put Weekly - Strike: {strike:.2f}, Expiración: {expiration}, Prima: {premium:.2f}"

    else:
        return "No operar opciones"

# ==================== FUNCIÓN REUSABLE PARA ANÁLISIS ====================
def analyze_dataframe(df: pd.DataFrame, timeframe_name: str, ticker: str, order_n: int = 5):
    print(f"  Timeframe: {timeframe_name}")
    close_last = df['Close'].iloc[-1]
    smas = {p: calcular_SMA(df['Close'], p).iloc[-1] for p in PERIODS if len(df) >= p}
    print("    SMA:")
    for p, val in smas.items():
        print(f"      {p}: {val:.2f}")
    emas = {p: EMA(df['Close'], p).iloc[-1] for p in PERIODS}
    print("    EMA:")
    for p, val in emas.items():
        print(f"      {p}: {val:.2f}")
    sma50_series = calcular_SMA(df['Close'], 50)
    sma200_series = calcular_SMA(df['Close'], 200)
    macd_line, signal_line, histogram, macd_slope = calcular_MACD(df['Close'])
    rsi = calcular_RSI_Wilder(df['Close'])
    rsi_last = rsi.iloc[-1]
    macd_last = macd_line.iloc[-1]
    signal_last = signal_line.iloc[-1]
    histogram_last = histogram.iloc[-1]
    upper_bb, middle_bb, lower_bb = calcular_bollinger_bands(df['Close'])
    upper_last = upper_bb.iloc[-1] if not upper_bb.empty else None
    middle_last = middle_bb.iloc[-1] if not middle_bb.empty else None
    lower_last = lower_bb.iloc[-1] if not lower_bb.empty else None
    print("    Bollinger Bands:")
    print(f"      Upper: {upper_last:.2f}" if upper_last is not None else "      Upper: N/A")
    print(f"      Middle: {middle_last:.2f}" if middle_last is not None else "      Middle: N/A")
    print(f"      Lower: {lower_last:.2f}" if lower_last is not None else "      Lower: N/A")
    rsi_sig = interpretar_rsi(rsi_last)
    macd_sig = interpretar_macd(macd_last, signal_last)
    ma_sig = interpretar_ma(close_last, smas.get(50, None), smas.get(200, None))
    cross_sig = interpretar_cross(sma50_series, sma200_series)
    bb_sig = interpretar_bollinger(close_last, upper_last, lower_last) if upper_last is not None and lower_last is not None else "neutral"
    print(f"    Bollinger Signal: {bb_sig}")
    pivotes = encontrar_pivotes(df)
    es_valido, patron_elliott = validar_impulso_elliott(pivotes)
    elliott_sig = "buy" if es_valido else "neutral"
    patron = analizar_patrones_recientes(df, close_last, order_n=order_n, plot=False)
    print(f"    RSI (último): {rsi_last:.2f} → {rsi_sig}")
    print(f"    MACD ahora: {macd_last:.4f}  Señal: {signal_last:.4f} Histograma: {histogram_last:.4f} → {macd_sig}")
    print(f"    MA (basado en SMA50 y SMA200): {ma_sig}")
    print(f"    Cross (Golden/Death): {cross_sig}")
    print(f"    Elliott Impulse: {elliott_sig}")
    print(f"    Patrón Detectado: {patron}")
    decision = get_decision(patron, es_valido, cross_sig, ma_sig, macd_last, signal_last, rsi_last, close_last, upper_last or 0, lower_last or 0)
    print(f"    Decisión (buy/hold/sell): {decision}")
    if timeframe_name == "Daily":
        # Obtener recomendación de opciones (solo para daily)
        calls, puts = get_weekly_options(ticker, END_DATE, close_last)
        option_rec = get_option_recommendation(decision, calls, puts, close_last)
        print(f"    Recomendación de Opciones: {option_rec}")
    print("----------------------------------------------------------------------")
    return decision, rsi_last, patron

# ==================== EJECUCIÓN ====================
PERIODS = [5, 10, 20, 50, 100, 200]

print("\n\n========================= RESUMEN DE INDICADORES CON INTERPRETACIÓN =========================\n")

results = []  # Para colectar y mostrar en tabla al final

for ticker in TICKERS:
    try:
        print(f"Ticker: {ticker}")
        df_daily = get_alpaca_data(ticker, START_DATE, END_DATE)
        df_weekly = resample_to_weekly(df_daily)
        daily_decision, daily_rsi, daily_patron = analyze_dataframe(df_daily, "Daily", ticker, order_n=5)
        weekly_decision, weekly_rsi, weekly_patron = analyze_dataframe(df_weekly, "Weekly", ticker, order_n=10)

        # Nueva lógica de decisión según nuevas reglas
        final_signal = 'HOLD'

        # Prioridad 1: Patrones Semanales (confiar más en semanales, ej. AMZN)
        if weekly_patron != 'NEUTRAL':
            if 'ALCISTA' in weekly_patron:
                final_signal = 'BUY'
            elif 'BAJISTA' in weekly_patron:
                final_signal = 'SELL'
            print(f"    Patrón Semanal detectado: {final_signal}")

        # Prioridad 2: Patrones Diarios (no olvidar patrones clave, ej. META)
        if final_signal == 'HOLD' and daily_patron != 'NEUTRAL':
            if 'ALCISTA' in daily_patron:
                final_signal = 'BUY'
            elif 'BAJISTA' in daily_patron:
                final_signal = 'SELL'
            print(f"    Patrón Diario detectado: {final_signal}")

        # Si aún HOLD, alineación de decisions con veto RSI
        if final_signal == 'HOLD':
            if daily_decision == 'buy' and weekly_decision == 'buy':
                candidate = 'BUY'
            elif daily_decision == 'sell' and weekly_decision == 'sell':
                candidate = 'SELL'
            else:
                candidate = 'HOLD'
            print(f"    Alineación de Timeframes: {candidate}")

            if candidate == 'HOLD':
                final_signal = 'HOLD'
            else:
                if candidate == 'BUY':
                    # RSI Overbought si >70 en cualquiera
                    if daily_rsi > 70 or weekly_rsi > 70:
                        final_signal = 'HOLD'
                        print("    Veto RSI aplicado: BUY cambiado a HOLD (Overbought)")
                    else:
                        final_signal = 'BUY'
                        print("    RSI OK: Mantener BUY")
                else:
                    # Para SELL, no hay veto según las reglas
                    final_signal = 'SELL'
                    print("    RSI OK: Mantener SELL")

        print(f"  Final Signal: {final_signal}")

        # Colectar para tabla
        results.append({
            'Ticker': ticker,
            'Daily Decision': daily_decision.upper(),
            'Weekly Decision': weekly_decision.upper(),
            'Final Signal': final_signal
        })

        print("======================================================================\n")
    except Exception as e:
        print(f"Error procesando {ticker}: {e}")

# Mostrar tabla final con señales
if results:
    df_results = pd.DataFrame(results)
    print("\n\n========================= TABLA DE SEÑALES FINALES =========================\n")
    print(df_results)



========================= RESUMEN DE INDICADORES CON INTERPRETACIÓN =========================

Ticker: AAPL
  Timeframe: Daily
    SMA:
      5: 269.86
      10: 265.51
      20: 258.51
      50: 248.25
      100: 229.95
      200: 223.30
    EMA:
      5: 268.86
      10: 265.39
      20: 260.41
      50: 249.19
      100: 237.66
      200: 228.57
    Bollinger Bands:
      Upper: 275.56
      Middle: 258.51
      Lower: 241.46
    Bollinger Signal: neutral
    RSI (último): 69.06 → caution_alcista
    MACD ahora: 6.2959  Señal: 5.3027 Histograma: 0.9931 → buy
    MA (basado en SMA50 y SMA200): buy
    Cross (Golden/Death): neutral
    Elliott Impulse: neutral
    Patrón Detectado: NEUTRAL
    Decisión (buy/hold/sell): hold
    Recomendación de Opciones: No operar opciones
----------------------------------------------------------------------
  Timeframe: Weekly
    SMA:
      5: 257.75
      10: 249.56
      20: 231.93
      50: 226.57
      100: 214.24
    EMA:
      5: 259.08
   